In [1]:
import sys
sys.path.append("../")

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot
from tqdm import tqdm
from tabulate import tabulate
import pickle as pkl
pd.set_option("display.max_columns", None)

In [3]:
class Data:
    
    def __init__(self, path):
        self.df = {
            'raw': pd.read_pickle(path)
        }
        if 'time' in self.df['raw'].columns:
            self.df['raw']['time'] = [ x.replace(tzinfo=None) for x in self.df['raw']['time']]

    def __repr__(self) -> str:
        repr = str()
        for name, df in self.df.items():
            repr = repr + name + ':\n' + str(df.head(3)) + '\n'
        return repr

    def shorten(self, name: str, rows: int, direction: int, source: str='raw', cols: list=None):
        '''Create new dataframe with specified list of columns and number of rows
        direction: 1 if data should be selected from top and -1 if from bottom
        '''
        assert (direction != 1 or direction != -1), 'direction must be 1 (top) or -1 (bottom)'
        
        if cols == None:
            cols = self.df[source].columns
        if direction == 1:
            self.df[name] = self.df[source][cols].iloc[:rows].copy()
        else:
            self.df[name] = self.df[source][cols].iloc[-rows:].copy()
        self.df[name].reset_index(drop=True, inplace=True)

    def add_columns(self, name: str, cols: list):
        '''Add new columns to component dataframes
        '''        
        exist_cols = list(self.df[name].columns)
        cols = exist_cols + cols
        self.df[name] = self.df[name].reindex(columns = cols) 

    def prepare_fast_data(self, name: str):
        '''Prepare data as an array for fast processing
        fcols = {col1: col1_index, col2: col2_index, .... }     
        fdata = [array[col1], array[col2], array[col3], .... ]
        Accessed by: self.fdata[fcols[column_name]] for whole column or
                     self.fdata[fcols[column_name]][row_index] for a specific row item
        '''
        self.fcols = dict()
        for i in range(len(self.df[name].columns)):
            self.fcols[self.df[name].columns[i]] = i
        self.fastdf = [self.df[name][col].array for col in self.df[name].columns]

    def fdata(self, column: str, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            return self.fastdf[self.fcols[column]]
        else:
            return self.fastdf[self.fcols[column]][index]
        
    def update_fdata(self, column: str, value, index: int=-1):
        assert index >= -1, 'Row index cannot be negative'
        if index == -1:
            self.fastdf[self.fcols[column]] = value
        else:
            self.fastdf[self.fcols[column]][index] = value


In [4]:
PATH = 'D:/Trading/forex_bot/outputs/'

In [5]:
files = ['inputs4.pkl', 'inputs5.pkl', 'inputs6.pkl']

In [6]:
dfs = list()
for f in files:
    dfs.append(pd.read_pickle(PATH + f))

In [7]:
inputs = pd.concat(dfs)
inputs.drop_duplicates(inplace=True)
inputs.reset_index(drop=True,inplace=True)
inputs

,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit
0,sim_3001,-1,10,2.0,0.002,3,True,11
1,sim_3002,-1,10,2.0,0.003,3,True,11
2,sim_3003,1,10,2.0,0.004,3,True,11
3,sim_3004,-1,50,2.0,0.002,3,True,11
4,sim_3005,-1,50,2.0,0.003,3,True,11
5,sim_3006,-1,50,2.0,0.004,3,True,11
6,sim_3007,1,100,2.0,0.002,3,True,11
7,sim_3008,1,100,2.0,0.003,3,True,11
8,sim_3009,1,100,2.0,0.004,3,True,11
9,sim_4001,1,10,2.0,0.002,3,True,12


In [8]:
inputs.to_pickle(PATH + 'inputs.n.all.pkl')

In [9]:
d = Data(PATH + "inputs.n.all.pkl")

In [10]:
subset = ['init_signal', 'base_trade_size', 'cushion', 'risk', 'rr', 'margin_closeout', 'streak_limit']
d.df['raw'].drop_duplicates(subset=subset, keep='first', inplace=True)
d.df['raw'].reset_index(drop=True, inplace=True)

In [11]:
cols = ['return_%', 'final_ac_bal', 'total_trades', 'avg_trade', 'avg_trade_duration', 'win_%', 'loss_%', 
        'expectancy', 'sharpe_ratio', 'sortino_ratio', 'calmar_ratio',
        'max_drawdown', 'max_ac_bal', 'min_ac_bal', 'max_margin_used', 'total_streaks', 'avg_trades_per_streak',
        'total_wins', 'max_win', 'avg_win', 'avg_win_duration', 'total_losses', 'max_loss', 'avg_loss', 'avg_loss_duration',
        'total_long', 'avg_long', 'avg_long_duration',
        'total_long_wins', 'avg_long_win', 'avg_long_win_duration',
        'total_long_losses', 'avg_long_loss', 'avg_long_loss_duration',
        'total_short', 'avg_short', 'avg_short_duration',
        'total_short_wins', 'avg_short_win', 'avg_short_win_duration',
        'total_short_losses', 'avg_short_loss', 'avg_short_loss_duration',
        
        ]
d.add_columns('raw', cols)
d.df['raw'].head(3)

,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_3001,-1,10,2.0,0.002,3,True,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sim_3002,-1,10,2.0,0.003,3,True,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,sim_3003,1,10,2.0,0.004,3,True,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
d.prepare_fast_data('raw')

In [13]:
d.fdata('sim_name', 0)

'sim_3001'

In [14]:
def convert2date(date_time):
    return date_time.date().strftime("%Y-%m-%d")

In [15]:
def sharpe_ratio(returns: pd.Series, risk_free_rate: float=0) -> float:
    return (returns.mean() - risk_free_rate) / returns.std()

def sortino_ratio(returns: pd.Series, risk_free_rate: float=0, target_return: float=0) -> float:
    downside_returns = np.array([ret - risk_free_rate for ret in returns if ret < target_return])
    std_dev_downside = np.std(downside_returns, ddof=1) if len(downside_returns) > 0 else 0
    return (np.mean(returns) - risk_free_rate) / std_dev_downside if std_dev_downside != 0 else np.nan

def max_drawdown(returns: pd.Series) -> float:
    '''
    cum_returns: Calculates the cumulative returns using np.cumprod(1 + returns).
    peaks: Finds the peaks in the cumulative returns using np.maximum.accumulate(cum_returns).
    drawdowns: Calculates the drawdowns for each point by subtracting the peaks from the cumulative returns and dividing by the peaks.
    max_drawdown: Identifies the maximum drawdown by finding the minimum drawdown value using np.min(drawdowns).'''
    cum_returns = np.cumprod(1 + returns.values)
    peaks = np.maximum.accumulate(cum_returns)
    drawdowns = (cum_returns - peaks) / peaks
    max_drawdown = np.min(drawdowns)
    return max_drawdown

def calmar_ratio(returns: pd.Series, risk_free_rate: float=0):
    max_drawdown_value = max_drawdown(returns)
    annual_return = np.mean(returns) * 311  # Assuming 311 trading days per year for forex. Its usually 252 for stocks
    return annual_return / abs(max_drawdown_value)

In [16]:
def populate_results(d: Data, i: int, df: pd.DataFrame):
    d.update_fdata('return_%', (df['ac_bal'].iloc[-1] - df['ac_bal'].iloc[0]) / df['ac_bal'].iloc[0], i)
    d.update_fdata('final_ac_bal', df['ac_bal'].iloc[-1], i)
    d.update_fdata('max_ac_bal', df['ac_bal'].max(), i)
    d.update_fdata('min_ac_bal', df['ac_bal'].min(), i)
    d.update_fdata('max_margin_used', df['margin_used'].max(), i)
    d.update_fdata('total_streaks', df['streak_no'].iloc[-1], i)
    d.update_fdata('avg_trades_per_streak', df.groupby('streak_no')['trade_no'].max().mean(), i)
    
    trades = df[df.signal != 0].copy()
    trades['return'] = trades['ac_bal'].shift(-1) - trades['ac_bal']
    trades['return_%'] = trades['ac_bal'].pct_change().shift(-1)
    trades['index'] = trades.index
    trades['duration'] = trades['index'].shift(-1) - trades['index']
    trades.dropna(inplace=True)
    # print(trades[['ac_bal', 'return', 'return_%']].head())

    d.update_fdata('sharpe_ratio', sharpe_ratio(trades['return_%']), i)
    d.update_fdata('sortino_ratio', sortino_ratio(trades['return_%']), i)
    d.update_fdata('calmar_ratio', calmar_ratio(trades['return_%']), i)
    d.update_fdata('max_drawdown', max_drawdown(trades['return_%']), i)

    d.update_fdata('total_trades', trades.shape[0], i)
    d.update_fdata('avg_trade', trades['return'].mean(), i)
    d.update_fdata('avg_trade_duration', trades['duration'].mean(), i)
    
    wins = trades[trades['return'] > 0]
    d.update_fdata('total_wins', wins.shape[0], i)
    d.update_fdata('max_win', wins['return'].max(), i)
    d.update_fdata('avg_win', wins['return'].mean(), i)
    d.update_fdata('avg_win_duration', wins['duration'].mean(), i)
    d.update_fdata('win_%', d.fdata('total_wins', i) / d.fdata('total_trades', i), i)

    losses = trades[trades['return'] < 0]
    d.update_fdata('total_losses', losses.shape[0], i)
    d.update_fdata('max_loss', losses['return'].min(), i)
    d.update_fdata('avg_loss', losses['return'].mean(), i)
    d.update_fdata('avg_loss_duration', losses['duration'].mean(), i)
    d.update_fdata('loss_%', d.fdata('total_losses', i) / d.fdata('total_trades', i), i)
    
    longs = trades[trades.signal == 1]
    d.update_fdata('total_long', longs.shape[0], i)
    d.update_fdata('avg_long', longs['return'].mean(), i)
    d.update_fdata('avg_long_duration', longs['duration'].mean(), i)

    long_wins = longs[longs['return'] > 0]
    d.update_fdata('total_long_wins', long_wins.shape[0], i)
    d.update_fdata('avg_long_win', long_wins['return'].mean(), i)
    d.update_fdata('avg_long_win_duration', long_wins['duration'].mean(), i)

    long_losses = longs[longs['return'] < 0]
    d.update_fdata('total_long_losses', long_losses.shape[0], i)
    d.update_fdata('avg_long_loss', long_losses['return'].mean(), i)
    d.update_fdata('avg_long_loss_duration', long_losses['duration'].mean(), i)

    shorts = trades[trades.signal == -1]
    d.update_fdata('total_short', shorts.shape[0], i)
    d.update_fdata('avg_short', shorts['return'].mean(), i)
    d.update_fdata('avg_short_duration', shorts['duration'].mean(), i)

    short_wins = shorts[shorts['return'] > 0]
    d.update_fdata('total_short_wins', short_wins.shape[0], i)
    d.update_fdata('avg_short_win', short_wins['return'].mean(), i)
    d.update_fdata('avg_short_win_duration', short_wins['duration'].mean(), i)

    short_losses = shorts[shorts['return'] < 0]
    d.update_fdata('total_short_losses', short_losses.shape[0], i)
    d.update_fdata('avg_short_loss', short_losses['return'].mean(), i)
    d.update_fdata('avg_short_loss_duration', short_losses['duration'].mean(), i)


    d.update_fdata('expectancy', d.fdata('win_%', i) * d.fdata('avg_win', i) + d.fdata('loss_%', i) * d.fdata('avg_loss', i), i)

In [17]:
sims = d.df['raw'].shape[0]
for i in tqdm(range(sims), desc=" Analysing... "):
# for i in tqdm(range(2), desc=" Analysing... "):
    data = pd.read_pickle(PATH + d.fdata('sim_name', i) + '.pkl')
    assert (d.fdata('sim_name', i) == data['sim_name'] and
            d.fdata('init_signal', i) == data['init_signal'] and
            d.fdata('base_trade_size', i) == data['base_trade_size'] and
            d.fdata('cushion', i) == data['cushion'] and
            d.fdata('risk', i) == data['risk'] and
            d.fdata('rr', i) == data['rr'] and
            d.fdata('margin_closeout', i) == data['margin_closeout'] and
            d.fdata('streak_limit', i) == data['streak_limit']), f"Parameters mismatch for {d.fdata('sim_name', i)}"
    
    populate_results(d, i, data['results'])

 Analysing... : 100%|██████████| 27/27 [00:03<00:00,  7.05it/s]


In [18]:
data

{'sim_name': 'sim_5009',
 'init_signal': -1,
 'base_trade_size': 100,
 'cushion': 2.0,
 'risk': 0.004,
 'rr': 3,
 'margin_closeout': True,
 'streak_limit': 13,
 'results':                       time    bid_o    bid_h    bid_l    bid_c    ask_o  \
 0      2016-01-07 00:00:00  1.07757  1.07802  1.07750  1.07777  1.07772   
 1      2016-01-07 00:05:00  1.07779  1.07811  1.07755  1.07802  1.07798   
 2      2016-01-07 00:10:00  1.07803  1.07823  1.07803  1.07819  1.07822   
 3      2016-01-07 00:15:00  1.07815  1.07821  1.07790  1.07790  1.07834   
 4      2016-01-07 00:20:00  1.07787  1.07789  1.07768  1.07782  1.07804   
 ...                    ...      ...      ...      ...      ...      ...   
 594298 2023-12-29 21:35:00  1.10355  1.10374  1.10355  1.10371  1.10374   
 594299 2023-12-29 21:40:00  1.10370  1.10379  1.10356  1.10371  1.10388   
 594300 2023-12-29 21:45:00  1.10371  1.10384  1.10371  1.10378  1.10388   
 594301 2023-12-29 21:50:00  1.10379  1.10389  1.10375  1.10383  1.10

In [19]:
with open(PATH + 'hedging_sim_analysis.n.pkl', 'wb') as file:
    pkl.dump(d.df['raw'], file)

In [20]:
d.df['raw'].head()

,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_3001,-1,10,2.0,0.002,3,True,11,-0.057775,942.22454,4968.0,-0.011630,119.554750,0.215781,0.784219,-0.011630,-0.021480,-0.039517,-0.051915,-0.070854,1002.22034,931.20928,23.600000,1133.0,4.385702,1072.0,8.79336,0.424225,223.084888,3896.0,-3.57540,-0.131557,91.068018,2483.0,-0.002535,114.602094,535.0,0.458308,209.887850,1948.0,-0.129101,88.432752,2485.0,-0.020717,124.503421,537.0,0.390268,236.232775,1948.0,-0.134012,93.703285
1,sim_3002,-1,10,2.0,0.003,3,True,11,-0.079735,920.26476,2273.0,-0.035079,261.448306,0.229212,0.770788,-0.035079,-0.056697,-0.086416,-0.129449,-0.087333,1000.08492,912.74466,23.600000,553.0,4.112116,521.0,7.50480,0.440882,441.865643,1752.0,-4.09224,-0.176618,207.796804,1138.0,-0.012442,248.965729,262.0,0.543101,421.377863,876.0,-0.178598,197.399543,1135.0,-0.057776,273.963877,259.0,0.337478,462.590734,876.0,-0.174638,218.194064
2,sim_3003,1,10,2.0,0.004,3,True,11,0.005429,1005.42937,1275.0,0.004258,465.818039,0.227451,0.772549,0.004258,0.004840,0.009181,0.039667,-0.037013,1037.67452,979.22459,23.600000,304.0,4.197368,290.0,9.01992,0.795488,770.944828,985.0,-7.35612,-0.228693,375.983756,638.0,0.027306,431.800940,145.0,0.880302,713.179310,493.0,-0.223575,349.042596,637.0,-0.018825,499.888540,145.0,0.710674,828.710345,492.0,-0.233820,402.979675
3,sim_3004,-1,50,2.0,0.002,3,True,11,-0.275124,724.87579,4968.0,-0.055379,119.554750,0.215781,0.784219,-0.055379,-0.019359,-0.036196,-0.055864,-0.334055,1010.42282,672.88611,111.566667,1133.0,4.385702,1072.0,41.56974,2.008414,223.084888,3896.0,-16.90235,-0.623240,91.068018,2483.0,-0.012493,114.602094,535.0,2.169178,209.887850,1948.0,-0.611668,88.432752,2485.0,-0.098231,124.503421,537.0,1.848248,236.232775,1948.0,-0.634812,93.703285
4,sim_3005,-1,50,2.0,0.003,3,True,11,-0.378302,621.69840,2273.0,-0.166433,261.448306,0.229212,0.770788,-0.166433,-0.055288,-0.084468,-0.152067,-0.414005,1000.38772,586.22174,111.566667,553.0,4.112116,521.0,35.47820,2.088745,441.865643,1752.0,-19.34566,-0.837065,207.796804,1138.0,-0.059510,248.965729,262.0,2.571627,421.377863,876.0,-0.846449,197.399543,1135.0,-0.273638,273.963877,259.0,1.600269,462.590734,876.0,-0.827681,218.194064


In [21]:
with open(PATH + 'hedging_sim_analysis.n.pkl', 'rb') as f:
    df_all = pkl.load(f)

In [22]:
df_all.drop_duplicates(subset=['init_signal', 'base_trade_size', 'cushion', 'risk', 'rr', 'margin_closeout', 'streak_limit'],keep='first')
df_all

,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,sim_3001,-1,10,2.0,0.002,3,True,11,-0.057775,942.22454,4968.0,-0.011630,119.554750,0.215781,0.784219,-0.011630,-0.021480,-0.039517,-0.051915,-0.070854,1002.22034,931.20928,23.600000,1133.0,4.385702,1072.0,8.79336,0.424225,223.084888,3896.0,-3.57540,-0.131557,91.068018,2483.0,-0.002535,114.602094,535.0,0.458308,209.887850,1948.0,-0.129101,88.432752,2485.0,-0.020717,124.503421,537.0,0.390268,236.232775,1948.0,-0.134012,93.703285
1,sim_3002,-1,10,2.0,0.003,3,True,11,-0.079735,920.26476,2273.0,-0.035079,261.448306,0.229212,0.770788,-0.035079,-0.056697,-0.086416,-0.129449,-0.087333,1000.08492,912.74466,23.600000,553.0,4.112116,521.0,7.50480,0.440882,441.865643,1752.0,-4.09224,-0.176618,207.796804,1138.0,-0.012442,248.965729,262.0,0.543101,421.377863,876.0,-0.178598,197.399543,1135.0,-0.057776,273.963877,259.0,0.337478,462.590734,876.0,-0.174638,218.194064
2,sim_3003,1,10,2.0,0.004,3,True,11,0.005429,1005.42937,1275.0,0.004258,465.818039,0.227451,0.772549,0.004258,0.004840,0.009181,0.039667,-0.037013,1037.67452,979.22459,23.600000,304.0,4.197368,290.0,9.01992,0.795488,770.944828,985.0,-7.35612,-0.228693,375.983756,638.0,0.027306,431.800940,145.0,0.880302,713.179310,493.0,-0.223575,349.042596,637.0,-0.018825,499.888540,145.0,0.710674,828.710345,492.0,-0.233820,402.979675
3,sim_3004,-1,50,2.0,0.002,3,True,11,-0.275124,724.87579,4968.0,-0.055379,119.554750,0.215781,0.784219,-0.055379,-0.019359,-0.036196,-0.055864,-0.334055,1010.42282,672.88611,111.566667,1133.0,4.385702,1072.0,41.56974,2.008414,223.084888,3896.0,-16.90235,-0.623240,91.068018,2483.0,-0.012493,114.602094,535.0,2.169178,209.887850,1948.0,-0.611668,88.432752,2485.0,-0.098231,124.503421,537.0,1.848248,236.232775,1948.0,-0.634812,93.703285
4,sim_3005,-1,50,2.0,0.003,3,True,11,-0.378302,621.69840,2273.0,-0.166433,261.448306,0.229212,0.770788,-0.166433,-0.055288,-0.084468,-0.152067,-0.414005,1000.38772,586.22174,111.566667,553.0,4.112116,521.0,35.47820,2.088745,441.865643,1752.0,-19.34566,-0.837065,207.796804,1138.0,-0.059510,248.965729,262.0,2.571627,421.377863,876.0,-0.846449,197.399543,1135.0,-0.273638,273.963877,259.0,1.600269,462.590734,876.0,-0.827681,218.194064
5,sim_3006,-1,50,2.0,0.004,3,True,11,0.133203,1133.20294,1273.0,0.104637,466.549882,0.230165,0.769835,0.104637,0.027032,0.061645,0.233027,-0.141193,1285.61467,987.39336,111.566667,305.0,4.177049,293.0,42.64078,3.749820,767.737201,980.0,-15.76437,-0.985198,376.501020,636.0,0.164991,430.941824,146.0,3.987137,720.116438,490.0,-0.973852,344.779592,637.0,0.044378,502.102041,147.0,3.514118,815.034014,490.0,-0.996544,408.222449
6,sim_3007,1,100,2.0,0.002,3,True,11,-0.546863,453.13743,4965.0,-0.110144,119.626989,0.215911,0.784089,-0.110144,-0.014408,-0.027744,-0.058513,-0.657261,1020.75201,349.85108,221.666667,1133.0,4.383054,1072.0,82.59300,3.984728,222.981343,3893.0,-33.58250,-1.237732,91.166709,2482.0,-0.035685,114.668815,535.0,4.258403,209.943925,1947.0,-1.215622,88.488957,2483.0,-0.184572,124.583166,537.0,3.712073,235.970205,1946.0,-1.259854,93.845838
7,sim_3008,1,100,2.0,0.003,3,True,11,-0.688197,311.80338,2268.0,-0.303438,262.024691,0.230600,0.769400,-0.303438,-0.047418,-0.072255,-0.188490,-0.769299,1046.03079,241.32081,221.666667,554.0,4.095668,523.0,70.49000,4.186368,441.541109,1745.0,-38.43700,-1.649093,208.221203,1136.0,-0.080632,249

In [23]:
df = df_all[df_all.margin_closeout == True].copy()
df.reset_index(inplace=True)
df

,index,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
0,0,sim_3001,-1,10,2.0,0.002,3,True,11,-0.057775,942.22454,4968.0,-0.011630,119.554750,0.215781,0.784219,-0.011630,-0.021480,-0.039517,-0.051915,-0.070854,1002.22034,931.20928,23.600000,1133.0,4.385702,1072.0,8.79336,0.424225,223.084888,3896.0,-3.57540,-0.131557,91.068018,2483.0,-0.002535,114.602094,535.0,0.458308,209.887850,1948.0,-0.129101,88.432752,2485.0,-0.020717,124.503421,537.0,0.390268,236.232775,1948.0,-0.134012,93.703285
1,1,sim_3002,-1,10,2.0,0.003,3,True,11,-0.079735,920.26476,2273.0,-0.035079,261.448306,0.229212,0.770788,-0.035079,-0.056697,-0.086416,-0.129449,-0.087333,1000.08492,912.74466,23.600000,553.0,4.112116,521.0,7.50480,0.440882,441.865643,1752.0,-4.09224,-0.176618,207.796804,1138.0,-0.012442,248.965729,262.0,0.543101,421.377863,876.0,-0.178598,197.399543,1135.0,-0.057776,273.963877,259.0,0.337478,462.590734,876.0,-0.174638,218.194064
2,2,sim_3003,1,10,2.0,0.004,3,True,11,0.005429,1005.42937,1275.0,0.004258,465.818039,0.227451,0.772549,0.004258,0.004840,0.009181,0.039667,-0.037013,1037.67452,979.22459,23.600000,304.0,4.197368,290.0,9.01992,0.795488,770.944828,985.0,-7.35612,-0.228693,375.983756,638.0,0.027306,431.800940,145.0,0.880302,713.179310,493.0,-0.223575,349.042596,637.0,-0.018825,499.888540,145.0,0.710674,828.710345,492.0,-0.233820,402.979675
3,3,sim_3004,-1,50,2.0,0.002,3,True,11,-0.275124,724.87579,4968.0,-0.055379,119.554750,0.215781,0.784219,-0.055379,-0.019359,-0.036196,-0.055864,-0.334055,1010.42282,672.88611,111.566667,1133.0,4.385702,1072.0,41.56974,2.008414,223.084888,3896.0,-16.90235,-0.623240,91.068018,2483.0,-0.012493,114.602094,535.0,2.169178,209.887850,1948.0,-0.611668,88.432752,2485.0,-0.098231,124.503421,537.0,1.848248,236.232775,1948.0,-0.634812,93.703285
4,4,sim_3005,-1,50,2.0,0.003,3,True,11,-0.378302,621.69840,2273.0,-0.166433,261.448306,0.229212,0.770788,-0.166433,-0.055288,-0.084468,-0.152067,-0.414005,1000.38772,586.22174,111.566667,553.0,4.112116,521.0,35.47820,2.088745,441.865643,1752.0,-19.34566,-0.837065,207.796804,1138.0,-0.059510,248.965729,262.0,2.571627,421.377863,876.0,-0.846449,197.399543,1135.0,-0.273638,273.963877,259.0,1.600269,462.590734,876.0,-0.827681,218.194064
5,5,sim_3006,-1,50,2.0,0.004,3,True,11,0.133203,1133.20294,1273.0,0.104637,466.549882,0.230165,0.769835,0.104637,0.027032,0.061645,0.233027,-0.141193,1285.61467,987.39336,111.566667,305.0,4.177049,293.0,42.64078,3.749820,767.737201,980.0,-15.76437,-0.985198,376.501020,636.0,0.164991,430.941824,146.0,3.987137,720.116438,490.0,-0.973852,344.779592,637.0,0.044378,502.102041,147.0,3.514118,815.034014,490.0,-0.996544,408.222449
6,6,sim_3007,1,100,2.0,0.002,3,True,11,-0.546863,453.13743,4965.0,-0.110144,119.626989,0.215911,0.784089,-0.110144,-0.014408,-0.027744,-0.058513,-0.657261,1020.75201,349.85108,221.666667,1133.0,4.383054,1072.0,82.59300,3.984728,222.981343,3893.0,-33.58250,-1.237732,91.166709,2482.0,-0.035685,114.668815,535.0,4.258403,209.943925,1947.0,-1.215622,88.488957,2483.0,-0.184572,124.583166,537.0,3.712073,235.970205,1946.0,-1.259854,93.845838
7,7,sim_3008,1,100,2.0,0.003,3,True,11,-0.688197,311.80338,2268.0,-0.303438,262.024691,0.230600,0.769400,-0.303438,-0.047418,-0.072255,-0.188490,-0.769299,1046.03079,241.32081,221.666667,554.0,4.095668,523.0,70.49000,4.186368,441.541109,1745.0,-38.43700,-1.649093,208.22120

In [24]:
df0 = df_all[df_all.margin_closeout == False].copy()
df0.reset_index(inplace=True)
df0

,index,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration


In [25]:
# figs1 = [
#             [px.scatter(df, x="cushion", y="return_%", color="init_signal", symbol="streak_limit"),
#             px.scatter(df, x="cushion", y="expectancy", color="init_signal", symbol="streak_limit")],
#             [px.scatter(df, x="cushion", y="win_%", color="init_signal", symbol="streak_limit"),
#             px.scatter(df, x="cushion", y="loss_%", color="init_signal", symbol="streak_limit")]
#     ]

# fig = make_subplots(
#     rows=2, cols=2,
#     subplot_titles=("return_%", "expectancy", "win_%", "loss_%"))

# for i, f1 in enumerate(figs1):
#     for j, f2 in enumerate(f1):
#         for trace in range(len(f2["data"])):
#             fig.append_trace(f2["data"][trace], row=i+1, col=j+1)
        
# fig.show()

In [26]:
px.scatter(df, x = 'rr', y = 'win_%', color = 'risk')

In [27]:
# win_% decreases with rr, lower rr higher win_%

In [28]:
px.scatter(df, x = 'risk', y = 'win_%', color='rr')

In [29]:
# win_% increases with risk if rr <= 2, higher risk higher win_% if rr <= 2

In [30]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'rr', facet_col='risk')

In [31]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'risk', facet_col='streak_limit')

In [32]:
# higher cushion, higher expectancy if risk >= 0.002

In [33]:
px.scatter(df, x = 'risk', y = 'expectancy', color = 'streak_limit', facet_col='streak_limit')

In [34]:
px.scatter(df, x = 'cushion', y = 'expectancy', color = 'streak_limit', facet_col='risk')

In [35]:
px.scatter(df, x = 'risk', y = 'max_loss')

In [36]:
# max loss decreases with risk, higher risk lower max loss

In [37]:
px.scatter(df, x='rr', y='max_loss', facet_col='streak_limit')

In [38]:
# max loss increases with lower rr but only for higher streak limits

In [39]:
px.scatter(df, x = 'risk', y = 'max_loss', color = 'streak_limit', facet_col='rr')

In [40]:
# max_loss decreases with risk, higher risk lower max loss

In [55]:
def plot_plots(df):  
    px.scatter(df, x = 'streak_limit', y = 'expectancy', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'return_%', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'max_margin_used', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'min_ac_bal', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'final_ac_bal', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'max_loss', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'max_win', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'sharpe_ratio', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'sortino_ratio', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'calmar_ratio', color = 'risk', facet_col='base_trade_size').show()
    px.scatter(df, x = 'streak_limit', y = 'max_drawdown', color = 'risk', facet_col='base_trade_size').show()

In [42]:
df1 = df[(df.risk>=0.003) & (df.rr <= 2) & (df.cushion==2)]
df1

,index,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration


In [56]:
plot_plots(df)

In [44]:
df2 = df[(df.risk>=0.003) & (df.rr > 2)]# & (df.cushion==2) & (df.streak_limit<=7)]
df2.head()

,index,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration
1,1,sim_3002,-1,10,2.0,0.003,3,True,11,-0.079735,920.26476,2273.0,-0.035079,261.448306,0.229212,0.770788,-0.035079,-0.056697,-0.086416,-0.129449,-0.087333,1000.08492,912.74466,23.600000,553.0,4.112116,521.0,7.50480,0.440882,441.865643,1752.0,-4.09224,-0.176618,207.796804,1138.0,-0.012442,248.965729,262.0,0.543101,421.377863,876.0,-0.178598,197.399543,1135.0,-0.057776,273.963877,259.0,0.337478,462.590734,876.0,-0.174638,218.194064
2,2,sim_3003,1,10,2.0,0.004,3,True,11,0.005429,1005.42937,1275.0,0.004258,465.818039,0.227451,0.772549,0.004258,0.004840,0.009181,0.039667,-0.037013,1037.67452,979.22459,23.600000,304.0,4.197368,290.0,9.01992,0.795488,770.944828,985.0,-7.35612,-0.228693,375.983756,638.0,0.027306,431.800940,145.0,0.880302,713.179310,493.0,-0.223575,349.042596,637.0,-0.018825,499.888540,145.0,0.710674,828.710345,492.0,-0.233820,402.979675
4,4,sim_3005,-1,50,2.0,0.003,3,True,11,-0.378302,621.69840,2273.0,-0.166433,261.448306,0.229212,0.770788,-0.166433,-0.055288,-0.084468,-0.152067,-0.414005,1000.38772,586.22174,111.566667,553.0,4.112116,521.0,35.47820,2.088745,441.865643,1752.0,-19.34566,-0.837065,207.796804,1138.0,-0.059510,248.965729,262.0,2.571627,421.377863,876.0,-0.846449,197.399543,1135.0,-0.273638,273.963877,259.0,1.600269,462.590734,876.0,-0.827681,218.194064
5,5,sim_3006,-1,50,2.0,0.004,3,True,11,0.133203,1133.20294,1273.0,0.104637,466.549882,0.230165,0.769835,0.104637,0.027032,0.061645,0.233027,-0.141193,1285.61467,987.39336,111.566667,305.0,4.177049,293.0,42.64078,3.749820,767.737201,980.0,-15.76437,-0.985198,376.501020,636.0,0.164991,430.941824,146.0,3.987137,720.116438,490.0,-0.973852,344.779592,637.0,0.044378,502.102041,147.0,3.514118,815.034014,490.0,-0.996544,408.222449
7,7,sim_3008,1,100,2.0,0.003,3,True,11,-0.688197,311.80338,2268.0,-0.303438,262.024691,0.230600,0.769400,-0.303438,-0.047418,-0.072255,-0.188490,-0.769299,1046.03079,241.32081,221.666667,554.0,4.095668,523.0,70.49000,4.186368,441.541109,1745.0,-38.43700,-1.649093,208.221203,1136.0,-0.080632,249.495599,263.0,5.159567,420.570342,873.0,-1.659295,197.957617,1132.0,-0.527030,274.598057,260.0,3.201939,462.753846,872.0,-1.638879,218.496560


In [45]:
plot_plots(df2)

In [46]:
df3 = df[(df.risk>=0.003) & (df.rr == 1.5) & (df.cushion==2) & (df.streak_limit==5)]
df3

,index,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration


In [47]:
def plot_plots(df):  
    px.scatter(df, x = 'init_signal', y = 'expectancy', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'return_%', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_margin_used', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'min_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'final_ac_bal', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_loss', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_win', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'sharpe_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'sortino_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'calmar_ratio', color = 'risk', facet_col='rr').show()
    px.scatter(df, x = 'init_signal', y = 'max_drawdown', color = 'risk', facet_col='rr').show()

In [48]:
plot_plots(df3)

win_% decreases with rr, lower rr higher win_%

win_% increases with risk, higher risk higher win_%

expectancy increases with risk, higher risk higher expectancy

init_signal has not effect on win_%


In [49]:
df1.sort_values(by='final_ac_bal')

,index,sim_name,init_signal,base_trade_size,cushion,risk,rr,margin_closeout,streak_limit,return_%,final_ac_bal,total_trades,avg_trade,avg_trade_duration,win_%,loss_%,expectancy,sharpe_ratio,sortino_ratio,calmar_ratio,max_drawdown,max_ac_bal,min_ac_bal,max_margin_used,total_streaks,avg_trades_per_streak,total_wins,max_win,avg_win,avg_win_duration,total_losses,max_loss,avg_loss,avg_loss_duration,total_long,avg_long,avg_long_duration,total_long_wins,avg_long_win,avg_long_win_duration,total_long_losses,avg_long_loss,avg_long_loss_duration,total_short,avg_short,avg_short_duration,total_short_wins,avg_short_win,avg_short_win_duration,total_short_losses,avg_short_loss,avg_short_loss_duration


In [50]:
df.calmar_ratio.max()

58.183786796999655

In [51]:
with open(PATH + 'sim_2250.pkl', 'rb') as f:
    tempdf = pkl.load(f)
tempdf = tempdf['results'].copy()

FileNotFoundError: [Errno 2] No such file or directory: 'D:/Trading/forex_bot/outputs/sim_2250.pkl'

In [ ]:
trades = tempdf[tempdf.signal!=0].copy()
trades.head()

In [ ]:
trades.ac_bal.pct_change().shift(-1)
px.histogram(x=trades.ac_bal.pct_change().shift(-1))

In [ ]:
returns = trades['ac_bal'].shift(-1) - trades['ac_bal']
px.histogram(x=returns)

In [ ]:
df.sort_values(by='final_ac_bal')